# Аналіз тонування тексту
Для аналізу тонування тексту певного твердження із соціальної мережі Twitter буде використовуватися логістична регресія. Дано твердження, потрібно вирішити чи має воно позитивне тонування чи негативне. Для цього потрібно зробити наступні кроки:  
* Визначити ознаки з деякого тексту для логістичної регресії
* Створити алгоритм логістичної регресії
* Застосувати логістичну регресію для задачі обробки натуральної мови
* Протестувати логістичну регресію
* Проаналізувати точність логістичної регресії

## 1) Імпорт функцій і даних

### Зчитування набору даних


Оригінальна документація набору вхідних приведена у [документація для набору даних із мережі Twitter](http://www.nltk.org/howto/twitter.html).

In [57]:
# run this cell to import nltk
import nltk
from os import getcwd

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Імпорт функцій

#### NaturalLanguageProcessing-модуль:
* process_tweet: повертає попередньо оброблене твердження, а саме видаляє усі посилання, хештег-символи, токенізує рядок, приводить слова до нижнього регістру, видаляє стоп слова та пунктуацію, приводить слова до кореня.
* build_freqs: повертає словник, що має (слово, тонація) в якості ключа і частоту використання слова до відповідної тональності в якості значення.


#### MachineLearning-модуль
* sigmoid: функція сигмоїди. Вихідним значенням є результат сигмоїди.
* gradientDescent: функція, що представляє собою алгоритм градієнтного спуску. У результаті виконання повертає значення функції вартості J та оновлений вектор вагових коефіцієнтів theta.
* extract_features: функція вилучення ознак у твердженні, що приймає одне твердження та словник частотності слів.
* predict_tweet: функція для прогнозування чи твердження позитивне чи негативне. Повертає прогнозоване значення за допомогою налаштованої моделі класифікації.
* test_logistic_regression: функція для перевірки точності побудованої моделі. Повертає значення точності прогнозування міток для тестового набору даних

In [58]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [59]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

from NaturalLanguageProcessing import process_tweet, build_freqs
from MachineLearning import sigmoid, gradientDescent, extract_features, predict_tweet, test_logistic_regression

## 2) Поділ на тренуючий та тестовий набори даних
Набір даних `twitter_samples` має підмножину з 5000 позитивних тверджень і 5000 негативних. Повна кількість записів у наборі даних складає 10000 тверджень.

Розбиття даних на тренуючий та тестовий набори відбуватися у наступних співвідношеннях: 
* 20% буде у тестовому наборі;
* 80% у тренуючому наборі.

In [60]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

train_pos = all_positive_tweets[:4000]
train_neg = all_negative_tweets[:4000]

test_pos = all_positive_tweets[4000:]
test_neg = all_negative_tweets[4000:]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

Вивід кількості даних у тренуючому та тестовому наборах відповідно.

In [61]:
print("Розмірність тренуючого набору даних = " + str(train_y.shape))
print("Розмірність тестуючого набору даних = " + str(test_y.shape))

Розмірність тренуючого набору даних = (8000, 1)
Розмірність тестуючого набору даних = (2000, 1)


### Приклад даних

In [62]:
import random
print('Твердження з позитивним змістом: \n' + train_pos[random.randint(0, 4000)])
print('\nТвердження з негативним змістом: \n' + train_neg[random.randint(0, 4000)])

Твердження з позитивним змістом: 
@VJAdeel 
Weltum he teh dety :p Huh =D

Твердження з негативним змістом: 
WHY MUST THE VIDEO STOP THO :(


## 3) Вилучення ознак

Для кожного твердження у наборі даних потрібно дістати дві ознаки, а саме:
* Перша ознака - це кількість позитивних слів у тверджені.
* Друга ознака - це кількість негативних слів у тверджені. 


### Етап попередньої обробки даних
Етап попередньої обробки даних відбувається разом з етапом вилучення ознак.
Функція для попередньої обробки даних `process_tweet`, що використовується у функції вилучення ознак `extract_features` містить усі етапи обробки даних:
* Токенізація рядка.
* Приведення тексту до нижнього регістру.
* Видалення стоп слів і пунктуації.
* Приведення слів до їх кореня.


### Приклад використання функції `process_tweet`

In [63]:
process_tweet_example = train_x[random.randint(0,8000)]
print('Твердження у соціальній мережі твітер: \n', process_tweet_example)
print('\nОброблений вид цього твердження: \n', process_tweet(process_tweet_example))

Твердження у соціальній мережі твітер: 
 @chingyapp hmmm, i think night better :)

Оброблений вид цього твердження: 
 ['hmmm', 'think', 'night', 'better', ':)']


### Вилучення ознак
#### Створення словника частотності
Створення словника частот наявних слів відбувається за допомогою імпортованої функції `build_freqs` .  
Ключом у такому словнику є пара (слово, мітка), наприклад ("дослідження", 1) або ("дослідження", 0). Значення, що зберігається для кожного ключа є кількість разів, коли слово "дослідження" асоціювалось з позитивної міткою, або з негативною міткою відповідно.

In [64]:
freqs = build_freqs(train_x, train_y)
print("Тип змінної, що представляє словник: " + str(type(freqs)))
print("Кількість слів у словнику: " + str(len(freqs.keys())))

Тип змінної, що представляє словник: <class 'dict'>
Кількість слів у словнику: 11436


#### Підрахунок частот
Для кожного слова у словника підраховується кількість вживання слова як у твердженнях з позитивною тональністю, так і з негативною. Таким чином, ми отримуємо дві характеристики.

In [81]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)
    

# training labels corresponding to X
Y = train_y

### Приклад використання функції `extract_features`

In [66]:
print('Приклад для твердження, що має слова наявні у словнику:\n')
extract_features_example = train_x[random.randint(0,8000)]
print('Твердження: ' + str(extract_features_example))
tmp1 = extract_features(extract_features_example, freqs)
print(tmp1)

print('\n\nПриклад для твердження, яке не має слів наявних у словнику:\n')
tmp1 = extract_features('Baaang HEEEY', freqs)
print(tmp1)

Приклад для твердження, що має слова наявні у словнику:

Твердження: @s0ulfl0wr When's your birthday ? :(
[[1.000e+00 5.400e+01 3.692e+03]]


Приклад для твердження, яке не має слів наявних у словнику:

[[1. 0. 0.]]


## 4) Побудова моделі передбачення 


### 4.1: Сигмоїда
Для класифікації тексту використовуватиметься логістична регресія, яка складається з сигмоїдної функції. 
* Сигмоїдна функція визначена як: 

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

Вона ставить у відповідність вхідній змінній z значення в інтервалі між 0 і 1, і таким чином може бути використана як функція вірогідності. 

<div style="width:image width px; font-size:100%; text-align:center;"><img src='./images/sigmoid_plot.jpg' alt="alternate text" width="width" height="height" style="width:300px;height:200px;" /> Figure 1 </div>

### 4.2 Логістична регресія: регресія і сигмоїда

Логістична регресія бере звичайну лінійну регресія і застосовує сигмоїду до вихідного значення лінійної регресії.

Регресія:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Тут  $\theta$ значення -  "вагові коефіцієнти".

Логістична регресія
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Тут змінна z за визначенням є 'логітом'.

### Частина 4.3: Функція вартості і градієнт
Функція вартості, що використовується для логістичної регресії розраховується як середнє арифметичне log-функцій для записів із навчального набору даних:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ - кількість записів із навчального набору даних (кількість навчальних записів)
* $y^{(i)}$ -  актуальна мітка для навчального запису 'i'.
* $h(z^{(i)})$ - прогнозована мітка для навчального запису 'i'.


Функція втрат для одного навчального запису:
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* Усі $h$ значення попадають у діапазон \[0, 1\], таким чином logs-функції будуть негативними. Це є причина чому мінусовий коефіцієнт був застосований до суми двох термів.

* Коли модель прогнозує одиницю ($h(z(\theta)) = 1$) і мітка 'y' також дорівнює одиниці, величина втрати для цього навчального запису дорівнює нулю. 
* Схожим чином, коли модель прогнозує нуль ($h(z(\theta)) = 0$) і актуальна мітка дорівнює нулю, величина втрат для цього навчального запису дорівнює нулю. 
* Але коли модель прогнозування близька до одиниці ($h(z(\theta)) = 0.9999$), а актуальна мітка дорівнює нулю, другий терм приймає велике негативне значення, яке потім помножено на мінусовий коефіцієнт -1 для конвертації у позитивне значення величини втрати. $-1 \times (1 - 0) \times log(1 - 0.9999) \approx 9.2$ Чим ближче модель прогнозує значення до 1, тим більша величина втрат.

In [67]:
-1 * (1 - 0) * np.log(1 - 0.9999) # loss is about 9.2

9.210340371976294

* Так само, якщо модель прогнозує значення близьке до нуля ($h(z) = 0.0001$) але актуальна мітка дорівнює одиниці, перший терм у функції втрат приймає велике значення: $-1 \times log(0.0001) \approx 9.2$.  Чим ближче модель прогнозує значення до нуля, тим більша величина втрат.

In [68]:
-1 * np.log(0.0001) # loss is about 9.2

9.210340371976182

### 4.4 Оновлення вагових коефіцієнтів


Щоб оновити вектор вагових коефіцієнтів $\theta$, потрібно застосувати алгоритм градієнтного спуску, щоб ітеративно покращувати прогнозування моделі. 

Градієнт функції вартості $J$ у відношенні до одного з вагових коефіцієнтів $\theta_j$ матиме вигляд:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x^{(i)}_j \tag{5}$$
* 'i' - це індекс поміж усіх 'm' навчальних записів.
* 'j' - це індекс вагового коефіцієнта $\theta_j$, таким чином $x^{(i)}_j$ - це ознака, що асоційована з ваговим коефіцієнтом $\theta_j$

* Щоб оновити ваговий коефіцієнт $\theta_j$, ми налаштовуємо його віднявши частку градієнта за допомогою коефіцієнта $\alpha$:
$$\theta_j = \theta_j - \alpha \times \nabla_{\theta_j}J(\theta) $$
* Темп навчання (англ. "learning rate") $\alpha$ - це значення, яке ми вибираємо для контроля наскільки великим буде значення одного оновлення.


### 4.5 Функція алгоритму градієнтного спуску
* Кількість ітерацій "num_iters" - це кількість разів, яка буде використана для навчального набору даних
* Для кожної ітерації буде обчислена функція вартості використовуючи усі навчальні записи (всього 'm' навчальних записів), і усі ознаки.
* Замість оновлення одного вагового коефіцієнта за одну одиницю часу, ми оновлюємо усі вагові коефіцієнти у векторі:
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\ 
\theta_2 
\\ 
\vdots
\\ 
\theta_n
\end{pmatrix}$$
* $\mathbf{\theta}$ має розмірність (n+1, 1), де 'n' - кількість ознак, а ще один елемент є вільним коефіцієнтом $\theta_0$ . Важливо зазначити, що відповідна ознака $\mathbf{x_0}$  має значення 1).
* 'логіти', 'z', обчислені за допомогою множення матриці ознак 'x' на вектор вагових коефіцієнтів 'theta'.  $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ має розмірність (m, n+1) 
    * $\mathbf{\theta}$: має розмірність (n+1, 1)
    * $\mathbf{z}$: має розмірність (m, 1)
* Прознозування 'h', обчислено за шляхом застосуванням сигмоїди до кожного елемента у 'z': $h(z) = sigmoid(z)$, і має розмірність (m,1).
* Функція вартості $J$ обчислена шляхом крапкового добутку векторів 'y' та 'log(h)'. Оскільки обидві змінні 'y' та 'h' є векторами розмірністю (m,1), треба транспонувати вектор, щоб множення вектора рядка матриці з колонкою вектора було крапковим добутком.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* Оновлення theta також векторізоване.  Оскільки розмірності  $\mathbf{x}$  (m, n+1), а обі змінні $\mathbf{h}$ і $\mathbf{y}$ (m, 1), потрібно транспонувати  $\mathbf{x}$ , щоб застосувати матричне множення, яке дасть  (n+1, 1) відповідь, що нам і потрібно:
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

Щоб натренувати модель потрібно Визвати функцію градієнтного спуску `gradientDescent`.


In [70]:
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"Функція вартості після тренування = {J:.8f}.")
print(f"Результуючий вектор вагових коефіцієнтів: {[round(t, 8) for t in np.squeeze(theta)]}")

Функція вартості після тренування = 0.22522315.
Результуючий вектор вагових коефіцієнтів: [6e-08, 0.00053818, -0.0005583]


## 5) Оцінка моделі на тестовому наборі даних

Після тренування моделі, перевіримо як наша модель може бути застосована на дійсних даних з якими модель ще не працювала.
Для цього викоростаємо функцію `predict_tweet`, яка із заданною побудованною моделю спрогнозує чи твердження позитивне або негативне

### Приклад використання функції `predict_tweet`

In [74]:
example_predict_tweet = ['I am happy',
                         'I am bad',
                         'this movie should have been great.',
                         'great',
                         'great great',
                         'great great great',
                         'great great great great',
                         'I am learning :)',
                        ':)',
                        ':(']

for tweet in example_predict_tweet:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))
    
    

I am happy -> 0.519275
I am bad -> 0.494347
this movie should have been great. -> 0.515979
great -> 0.516065
great great -> 0.532096
great great great -> 0.548062
great great great great -> 0.563929
I am learning :) -> 0.831103
:) -> 0.830885
:( -> 0.113930


### Оцінка моделі на тестовому наборі даних

Дано тестовий набір даних і вагові коефіцієнти побудованої моделі. Обчислимо точність побудованої логістичної регресійної моделі за допомогою функції `test_logistic_regression` , яка містить функцію для прогнозування кожного твердження у наборі даних - `predict_tweet`.
* Якщо прогнозування більше > 0.5, то встановити класифікацію моделі рівній 1, інакше встановити класифікацію моделі рівній 0.
* Прогнозування точне коли пронозована мітка відповідає актуальній. Потрібно підсумувати кількість правильних відповідностей і поділити на загальну кількість міток у тестовому наборі даних.


In [75]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Точність логістичної регресійної моделі = {tmp_accuracy:.4f}")

Точність логістичної регресійної моделі = 0.9950


## 6) Аналіз помилок

У цій частині буде видно деякі твердження, що були невірно класифіковані.

In [76]:
# Some error analysis done for you
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('Твердження:', x)
        print('Оброблене твердження:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Твердження: @MarkBreech Not sure it would be good thing 4 my bottom daring 2 say 2 Miss B but Im gonna be so stubborn on mouth soaping ! #NotHavingit :p
Оброблене твердження: ['sure', 'would', 'good', 'thing', '4', 'bottom', 'dare', '2', 'say', '2', 'miss', 'b', 'im', 'gonna', 'stubborn', 'mouth', 'soap', 'nothavingit', ':p']
1	0.48901497	b'sure would good thing 4 bottom dare 2 say 2 miss b im gonna stubborn mouth soap nothavingit :p'
Твердження: I'm playing Brain Dots : ) #BrainDots
http://t.co/UGQzOx0huu
Оброблене твердження: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48418949	b"i'm play brain dot braindot"
Твердження: I'm playing Brain Dots : ) #BrainDots http://t.co/aOKldo3GMj http://t.co/xWCM9qyRG5
Оброблене твердження: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.48418949	b"i'm play brain dot braindot"
Твердження: I'm playing Brain Dots : ) #BrainDots http://t.co/R2JBO8iNww http://t.co/ow5BBwdEMY
Оброблене твердження: ["i'm", 'play', 'brain', 'dot', 'braindot']
1	0.484189

Later in this specialization, we will see how we can use deeplearning to improve the prediction performance.

## 7) Прогнозування для свого власного твердження

In [80]:
# Feel free to change the tweet below
my_tweet = 'I hate when our football team plays on own game field!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['hate', 'footbal', 'team', 'play', 'game', 'field']
[[0.49534019]]
Negative sentiment
